In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
def get_item_page(items_url):
    # Download the page
    response = requests.get(items_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(items_url))
    # Parse using Beautiful soup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [4]:
#Now we have a summarised  steps in a function as below
def get_item_list_tags():
    item_list_tags = []
    for page in range(1,5):
          items_url = f"https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab={page}"
          response = requests.get(items_url)
          page_contents = response.text
          if response.status_code != 200:
              raise Exception('Failed to load page {}'.format(items_url))
          doc = BeautifulSoup(page_contents, "html.parser")
          for item in doc.find_all("div", {'class': "organic-gallery-offer-outter J-offer-wrapper"}):
                item_list_tags.append(item)
          sleep(randint(2,10))
          print('Downloading page number', page)  
    return item_list_tags
item_list_tags= get_item_list_tags()
len (item_list_tags)

32

In [5]:
#now we will do the  steps to get header details in a small function
def get_itemdetail(item_list_tags):
    all_item_titledetail_list = []
    for item in item_list_tags:
        title = item.find ('p', {'class': 'elements-title-normal__content medium'})
        all_item_titledetail_list.append({"Title":title.text.strip()})

    itemdetail=[]
    for tag in all_item_titledetail_list:
        itemdetail.append(tag.get('Title'))
    return itemdetail

itemdetail = get_itemdetail(item_list_tags)
len (itemdetail)#this will give length
itemdetail #this will list out the header details

['Most popular items neck strap lanyard chain vaper lanyard electronic atomizer lanyard for leisure time',
 'Fashionable inflight items plastic sound insulation ear plugs',
 'Cheap Custom Promotional Item,Promotional Product,Promotional Item China',
 'Most Popular Promotional Products and Corporate Gift Items to Build Brand',
 'One stop OEM & ODM Custom Logo Promotion Material Election Gadgets Political Campaign Vote Promotional Giveaways Gifts Items',
 '2021 Best selling Gift items custom promotional products for event giveaways',
 'SUNJET New Products Custom Mini Electronic Gift Items Led Fan',
 'RECOMMEND Best custom promotional Advertising items and giveaways Merchandise Products',
 'Most popular items neck strap lanyard chain vaper lanyard electronic atomizer lanyard for leisure time',
 'Items Promotional Items Custom Fashion Printing Charm Plastic High Quality Stand Up Transparent Acrylic Photo Keychain',
 'Fashionable inflight items plastic sound insulation ear plugs',
 'Cheap C

In [6]:
#we will summarise the above steps to get price in a function
def get_itemprice(item_list_tags):
    item_price_tags = []
    for item in item_list_tags:
        item_price = item.find ('span', {'class': 'elements-offer-price-normal__price'})
        if item_price == None:
            item_price_tags.append({"ItemPrice":item_price})
        else:
            item_price_tags.append({"ItemPrice":item_price.get_text(strip=True)})

    itemprice=[]
    for tag in item_price_tags:
        itemprice.append(tag.get('ItemPrice'))
    return itemprice

#this name "itemprice" should match with the name used in key:value pair value which will be used later in dict for df
itemprice = get_itemprice(item_list_tags)
len(itemprice)

32

In [7]:
#we will make a small function to get the rating/stars 
def get_Stars(item_list_tags):
    star_tags = []
    for item in item_list_tags:
        star = len(item.find_all('i', {'class': "iconfont iconzuanshi seller-star-level__dm dm-orange"}))
        star_tags.append({"Stars":star})
    star=[]
    for tag in star_tags:
        star.append(tag.get('Stars'))
    return star
star = get_Stars(item_list_tags)
star #this will list out the star details

[1,
 0,
 3,
 3,
 3,
 3,
 5,
 3,
 1,
 0,
 0,
 3,
 3,
 3,
 3,
 5,
 1,
 0,
 0,
 3,
 2,
 4,
 5,
 2,
 0,
 3,
 0,
 5,
 2,
 2,
 2,
 4]

In [8]:
#we will  get URLs in a small function
def get_URLS(item_list_tags):
    website_tags = []
    base_url = 'https:'
    for item in item_list_tags:
        website = item.find ('a', {'class': 'organic-gallery-offer__img-section'})
        website_tags.append({"Website":base_url+website['href']})
    URLS=[]
    for tag in website_tags:
        URLS.append(tag.get('Website'))
    return URLS
URLS = get_URLS(item_list_tags)

In [9]:
Dict={"Title":itemdetail,"ItemPrice":itemprice,"Stars":star,"Website":URLS}

In [10]:
#Combining everything together
def get_web_scraping():
    get_item_list_tags()

    Dict1 = {
        'title': get_itemdetail(item_list_tags),
        'itemprice': get_itemprice(item_list_tags),
        'starrs': get_Stars(item_list_tags),
        'url': get_URLS(item_list_tags)
    }
    return pd.DataFrame(Dict1)

function_based_df = get_web_scraping()
function_based_df[:5] #this will display the dataframe

,title,itemprice,starrs,url
0,Most popular items neck strap lanyard chain va...,$0.90,1,https://www.alibaba.com/product-detail/Most-po...
1,Fashionable inflight items plastic sound insul...,$0.038-$0.25,0,https://www.alibaba.com/product-detail/Fashion...
2,"Cheap Custom Promotional Item,Promotional Prod...",$0.10-$0.50,3,https://www.alibaba.com/product-detail/Cheap-C...
3,Most Popular Promotional Products and Corporat...,$15.00-$20.00,3,https://www.alibaba.com/product-detail/Most-Po...
4,One stop OEM & ODM Custom Logo Promotion Mater...,$0.42-$1.23,3,https://www.alibaba.com/product-detail/One-sto...
